In [1]:
import optuna
import os
import torch
import pytorch_lightning as pl

from optuna.visualization import plot_optimization_history, plot_intermediate_values, plot_parallel_coordinate, plot_contour, plot_slice, plot_param_importances
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning import Callback

In [2]:
from dataset.dataset import LFWDataset
from models.face_net import LightningFaceNet
from torch.utils.data import DataLoader, random_split

In [3]:
root = os.path.join('.', 'data', 'lfw-deepfunneled')
dataset = LFWDataset(root)

In [4]:
train_length = int(0.8 * len(dataset))
val_length = test_length = (len(dataset) - train_length) // 2

train_set, val_set, test_set = random_split(dataset, [train_length, val_length, test_length])

train_loader = DataLoader(train_set, batch_size=8, num_workers=0)
val_loader = DataLoader(val_set, batch_size=8, num_workers=0)
test_loader = DataLoader(test_set, batch_size=8, num_workers=0)

In [5]:
hparams = {'lr': 0.001, 'margin': 1}
model = LightningFaceNet(hparams)

trainer = pl.Trainer(gpus=1, max_epochs=2)

In [6]:
PERCENT_TEST_EXAMPLES = 0.1
EPOCHS = 10

class MetricsCallback(Callback):
    """PyTorch Lightning metric callback."""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)


def objective(trial):
    
    hparams = { 'lr': trial.suggest_loguniform('lr', 1e-5, 1000), 
                'margin': trial.suggest_loguniform('margin', 0.1, 2),
                'optimizer': trial.suggest_categorical('optimizer', ['SGD', 'Adam'])}
    
    metrics_callback = MetricsCallback()
    trainer = pl.Trainer(
        logger=False,
        max_epochs=EPOCHS,
        gpus=1 if torch.cuda.is_available() else None,
        callbacks=[metrics_callback, PyTorchLightningPruningCallback(trial, monitor="val_loss"),],
    )
    
    model = LightningFaceNet(hparams)
    trainer.fit(model, train_loader, val_loader)
    
    return metrics_callback.metrics[-1]["val_loss"].item()

In [7]:
pruner = optuna.pruners.NopPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=20)

[I 2020-12-14 22:09:08,365] A new study created in memory with name: no-name-36bdbd0b-cf51-4622-a7e5-4982dcbeb3b4
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 22:21:51,857] Trial 0 finished with value: 0.31600797176361084 and parameters: {'lr': 8.39126927699621e-05, 'margin': 0.6556643856373062, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.31600797176361084.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 22:37:21,197] Trial 1 finished with value: 0.10210084915161133 and parameters: {'lr': 0.1488746091356167, 'margin': 0.8972292139919623, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.31600797176361084.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 22:57:41,698] Trial 2 finished with value: 0.32961320877075195 and parameters: {'lr': 1.037361591155067e-05, 'margin': 0.321575065019838, 'optimizer': 'SGD'}. Best is trial 2 with value: 0.32961320877075195.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 23:09:19,377] Trial 3 finished with value: 0.0958993136882782 and parameters: {'lr': 222.35402981995645, 'margin': 0.169782926361717, 'optimizer': 'SGD'}. Best is trial 2 with value: 0.32961320877075195.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 23:21:11,780] Trial 4 finished with value: 0.3776259422302246 and parameters: {'lr': 102.85504607753919, 'margin': 1.959856723159538, 'optimizer': 'SGD'}. Best is trial 4 with value: 0.3776259422302246.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 23:34:37,670] Trial 5 finished with value: 0.5574241876602173 and parameters: {'lr': 0.0022220902921328664, 'margin': 1.5350862624250827, 'optimizer': 'Adam'}. Best is trial 5 with value: 0.5574241876602173.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[I 2020-12-14 23:49:41,806] Trial 6 finished with value: 0.11260208487510681 and parameters: {'lr': 0.2313560273976035, 'margin': 0.3307193879148246, 'optimizer': 'Adam'}. Best is trial 5 with value: 0.5574241876602173.
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | FaceNet | 40.5 M
----------------------------------
40.5 M    Trainable params
0         Non-trainable params
40.5 M    Total params


[W 2020-12-14 23:50:47,384] Trial 7 failed because of the following error: KeyError('val_loss')
Traceback (most recent call last):
  File "c:\users\thien\.virtualenvs\ibm-labcourse-jcrwhed5\lib\site-packages\optuna\_optimize.py", line 189, in _run_trial
    value = func(trial)
  File "<ipython-input-6-66a000397ee5>", line 32, in objective
    return metrics_callback.metrics[-1]["val_loss"].item()
KeyError: 'val_loss'


KeyError: 'val_loss'